In [1]:
# format data from html

import pandas as pd

from bs4 import BeautifulSoup
# soup = BeautifulSoup(html_doc, 'html.parser')
import os
os.listdir('./data')

['data_synop',
 'donnees-synop-essentielles-omm.csv',
 'fires_with_meteo.csv',
 'fires_with_meteo_full_one.csv',
 'liste_incendies_ du_10_01_2020.csv',
 'meteo_stations.csv',
 'private_test.csv',
 'private_train.csv',
 'synop_aglomerated.csv',
 'test.csv',
 'train.csv']

In [2]:
df = pd.read_csv("./data/meteo_stations.csv", sep=";")

In [3]:
parse = lambda x: BeautifulSoup(x, 'html.parser') 
def parse_(string: str):
    res = dict()
    for elt in BeautifulSoup(str(string), 'html.parser').find_all('li'):
        splitted = [sub.strip() for sub in elt.text.split(':')]
        res[splitted[0]] = splitted[1]
    return res
df['soups'] = df['response']

In [4]:
from tqdm import tqdm
parseds = list()
for idx, row in tqdm(df.iterrows(), total=len(df)):
    parseds.append(parse_(row['soups']))

100%|█████████████████████████████████████████████████████████████████████████| 206316/206316 [16:49<00:00, 204.33it/s]


In [5]:
to_add = pd.io.json.json_normalize(parseds)

In [13]:
total = pd.merge(df, to_add, left_index=True, right_index=True)

In [25]:
total.tail()

,date,response,insee_code,soups,Durée d'ensoleillement de la journée,Hauteur des précipitations,Température minimale de la journée,Température maximale de la journée
206311,2020-01-23,"<article class=""k-article report""> <header> <p...",2B033,"<article class=""k-article report""> <header> <p...",0h,0.8mm,5.4°C,13.9°C
206312,2020-01-24,"<article class=""k-article report""> <header> <p...",2B033,"<article class=""k-article report""> <header> <p...",0h,45.0mm,9.8°C,14.1°C
206313,2020-01-25,"<article class=""k-article report""> <header> <p...",2B033,"<article class=""k-article report""> <header> <p...",5h,0.0mm,6.7°C,16.0°C
206314,2020-01-26,"<article class=""k-article report""> <header> <p...",2B033,"<article class=""k-article report""> <header> <p...",7h,0.0mm,7.0°C,15.5°C
206315,2020-01-27,"<article class=""k-article report""> <header> <p...",2B033,"<article class=""k-article report""> <header> <p...",7h,0.0mm,5.5°C,15.5°C


In [26]:
total.drop(columns=['response', 'soups']).to_csv('./data/processed_meteo_stations.csv',sep=";")